# Credit Default Swap Pricing

This notebook replicates the QuantLib `CDS` example: bootstrap a hazard rate curve from CDS spreads and reprice the calibration instruments.

In [1]:
import pyquantlib as ql
from datetime import date

print(f"PyQuantLib {ql.__version__} (QuantLib {ql.QL_VERSION})")

PyQuantLib 0.1.0 (QuantLib 1.40)


## Market Data

From Lehman Brothers "Guide to Exotic Credit Derivatives": a zero flat yield curve
with flat CDS spreads of 150bp and recovery rate of 50% corresponds to a flat 3%
hazard rate.

In [2]:
calendar = ql.TARGET()
todays_date = date(2007, 5, 15)
todays_date = calendar.adjust(todays_date)
ql.Settings.evaluationDate = todays_date

# Flat yield curve at 1%
flat_rate = ql.SimpleQuote(0.01)
ts_curve = ql.FlatForward(todays_date, ql.QuoteHandle(flat_rate), ql.Actual365Fixed())
ts_handle = ql.YieldTermStructureHandle(ts_curve)

# CDS market
settlement_days = 1
recovery_rate = 0.5
quoted_spreads = [0.0150, 0.0150, 0.0150, 0.0150]
tenors = [ql.Period("3M"), ql.Period("6M"), ql.Period("1Y"), ql.Period("2Y")]

settlement_date = calendar.advance(todays_date, settlement_days, ql.Days)

print(f"Today:      {todays_date}")
print(f"Settlement: {settlement_date}")
print(f"Recovery:   {recovery_rate:.0%}")
print(f"Spreads:    {quoted_spreads[0]:.0%} flat")

Today:      May 15th, 2007
Settlement: May 16th, 2007
Recovery:   50%
Spreads:    2% flat


## Bootstrapping Hazard Rates

Build a piecewise default probability curve from the CDS spread quotes.

In [3]:
instruments = []
for i in range(4):
    helper = ql.SpreadCdsHelper(
        quoted_spreads[i], tenors[i], settlement_days, calendar,
        ql.Quarterly, ql.Following, ql.DateGeneration.TwentiethIMM,
        ql.Actual365Fixed(), recovery_rate, ts_handle,
    )
    instruments.append(helper)

hazard_curve = ql.PiecewiseBackwardFlatHazard(
    todays_date, instruments, ql.Actual365Fixed())

print("Calibrated hazard rate values:")
for d, v in hazard_curve.nodes():
    print(f"  {d}  {v:.6f}")

Calibrated hazard rate values:
  May 15th, 2007  0.029968
  September 20th, 2007  0.029968
  December 20th, 2007  0.029961
  June 20th, 2008  0.029962
  June 22nd, 2009  0.029961


## Survival Probabilities

In [4]:
sp_1y = hazard_curve.survivalProbability(
    calendar.advance(todays_date, 1, ql.Years))
sp_2y = hazard_curve.survivalProbability(
    calendar.advance(todays_date, 2, ql.Years))

print(f"1Y survival probability: {sp_1y:.4%}  (expected ~97.04%)")
print(f"2Y survival probability: {sp_2y:.4%}  (expected ~94.18%)")

1Y survival probability: 97.0401%  (expected ~97.04%)
2Y survival probability: 94.1758%  (expected ~94.18%)


## Repricing Calibration Instruments

Price CDS contracts at each tenor using the `MidPointCdsEngine` and the bootstrapped
hazard rate curve. The fair spreads should match the input market quotes.

In [5]:
prob_handle = ql.DefaultProbabilityTermStructureHandle(hazard_curve)
engine = ql.MidPointCdsEngine(prob_handle, recovery_rate, ts_handle)

nominal = 1_000_000.0
maturities = [calendar.adjust(settlement_date + t) for t in tenors]

print(f"{'Tenor':>6s}  {'Fair Spread':>12s}  {'NPV':>12s}  {'Default Leg':>12s}  {'Coupon Leg':>12s}")
print("-" * 60)

for i, (tenor, spread) in enumerate(zip(tenors, quoted_spreads)):
    schedule = ql.Schedule(
        settlement_date, maturities[i], ql.Period(ql.Quarterly),
        calendar, ql.Following, ql.Unadjusted,
        ql.DateGeneration.TwentiethIMM, False,
    )
    cds = ql.CreditDefaultSwap(
        ql.ProtectionSide.Seller, nominal, spread, schedule,
        ql.Following, ql.Actual365Fixed(),
    )
    cds.setPricingEngine(engine)
    print(f"{str(tenor):>6s}  {cds.fairSpread():12.6%}  {cds.NPV():12.2f}  {cds.defaultLegNPV():12.2f}  {cds.couponLegNPV():12.2f}")

 Tenor   Fair Spread           NPV   Default Leg    Coupon Leg
------------------------------------------------------------
    3M     1.500000%          0.00      -5177.05       5177.05
    6M     1.500000%          0.00      -8841.72       8841.72
    1Y     1.500000%         -0.00     -16101.81      16101.81
    2Y     1.500000%         -0.00     -30154.50      30154.50
